## Transform Orders Data - Explode Arrays
1. Access elements from the JSON object
1. Deduplicate Array Elements
1. Explode Arrays
1. Write the Transformed Data to Silver Schema

In [0]:
-- 1. 
select * from gizmobox.silver.orders_json